In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import sys

In [2]:
from os import path
import random

In [3]:
from scipy.sparse import csc_matrix, csr_matrix, save_npz
from scipy.io import mmread, mmwrite

# Read data

In [4]:
folder = '/gpfs/gibbs/pi/zhao/xs272/prep_pipeline/BMMC'

In [5]:
adata = sc.read_h5ad('data/BMMC/GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad')

In [6]:
adata

AnnData object with n_obs × n_vars = 69249 × 129921
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [7]:
adata.obs.cell_type.value_counts()

CD8+ T                 11589
CD14+ Mono             10843
NK                      6929
CD4+ T activated        5526
Naive CD20+ B           5052
Erythroblast            4916
CD4+ T naive            4398
Transitional B          2810
Proerythroblast         2300
CD16+ Mono              1894
B1 B                    1890
Normoblast              1780
Lymph prog              1779
G/M prog                1203
pDC                     1191
HSC                     1072
CD8+ T naive            1012
MK/E prog                884
cDC2                     859
ILC                      835
Plasma cell              379
ID2-hi myeloid prog      108
Name: cell_type, dtype: int64

In [8]:
# Separate ATAC and RNA data
is_gex_col = adata.var.feature_types == 'GEX'
adata_rna = adata[:, is_gex_col]
adata_atac = adata[:, ~is_gex_col]

In [9]:
adata_rna

View of AnnData object with n_obs × n_vars = 69249 × 13431
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [10]:
adata_atac

View of AnnData object with n_obs × n_vars = 69249 × 116490
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [11]:
# Get single RNA data
batch_rna = ['s1d2', 's1d3', 's3d3', 's4d9', 's3d10']
adata_rna_single = adata_rna[adata_rna.obs.batch.isin(batch_rna)]

In [12]:
adata_rna_single

View of AnnData object with n_obs × n_vars = 26450 × 13431
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [13]:
# Get single ATAC data
batch_atac = ['s2d4', 's2d5', 's3d7', 's4d8'] # s3d6 is not used because its fragment files are missing
adata_atac_single = adata_atac[adata_atac.obs.batch.isin(batch_atac)]

In [14]:
adata_atac_single

View of AnnData object with n_obs × n_vars = 22653 × 116490
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [15]:
# Get multiome RNA data
batch_rna = ['s1d1', 's2d1', 's4d1']
adata_rna_mul = adata_rna[adata_rna.obs.batch.isin(batch_rna)]
# Get multiome ATAC data
batch_atac = ['s1d1', 's2d1', 's4d1']
adata_atac_mul = adata_atac[adata_atac.obs.batch.isin(batch_atac)]

In [16]:
# del unnecessary data
del adata, adata_rna, adata_atac

## Save unimodal RNA files

In [17]:
rna_counts = csr_matrix(adata_rna_single.layers['counts'])
rna_features = adata_rna_single.var_names
rna_cells = adata_rna_single.obs_names
rna_annotation = np.array(adata_rna_single.obs.cell_type)
rna_path = path.join(folder, 'RNA')
np.savetxt(path.join(rna_path, 'cells.txt'), rna_cells, fmt='%s')
np.savetxt(path.join(rna_path, 'genes.txt'), rna_features, fmt='%s')
np.savetxt(path.join(rna_path, 'annotations.txt'), rna_annotation, fmt='%s')
mmwrite(path.join(rna_path, 'counts.mtx'), rna_counts, field = 'integer')

## Save unimodal ATAC files

In [21]:
atac_counts = csr_matrix(adata_atac_single.layers['counts'])
atac_features = adata_atac_single.var_names
atac_cells = adata_atac_single.obs_names
atac_annotation = np.array(adata_atac_single.obs.cell_type)
atac_path = path.join(folder, 'ATAC')
np.savetxt(path.join(atac_path, 'cells.txt'), atac_cells, fmt='%s')
np.savetxt(path.join(atac_path, 'genes.txt'), atac_features, fmt='%s')
np.savetxt(path.join(atac_path, 'annotations.txt'), atac_annotation, fmt='%s')
mmwrite(path.join(atac_path, 'counts.mtx'), atac_counts, field = 'integer')

In [37]:
# atac_counts = csr_matrix(adata_atac_single.obsm['ATAC_gene_activity'])
# atac_features = adata_atac_single.uns['ATAC_gene_activity_var_names']
# atac_cells = adata_atac_single.obs_names
# atac_annotation = np.array(adata_atac_single.obs.cell_type)
# atac_path = path.join(folder, 'ATAC.gene.activities')
# np.savetxt(path.join(atac_path, 'cells.txt'), atac_cells, fmt='%s')
# np.savetxt(path.join(atac_path, 'genes.txt'), atac_features, fmt='%s')
# np.savetxt(path.join(atac_path, 'annotations.txt'), atac_annotation, fmt='%s')
# mmwrite(path.join(atac_path, 'counts.mtx'), atac_counts, field = 'integer')

## Save multiome RNA files

In [23]:
rna_counts = csr_matrix(adata_rna_mul.layers['counts'])
rna_features = adata_rna_mul.var_names
rna_cells = adata_rna_mul.obs_names
rna_annotation = np.array(adata_rna_mul.obs.cell_type)
rna_path = path.join(folder, 'multiome/RNA')
np.savetxt(path.join(rna_path, 'cells.txt'), rna_cells, fmt='%s')
np.savetxt(path.join(rna_path, 'genes.txt'), rna_features, fmt='%s')
np.savetxt(path.join(rna_path, 'annotations.txt'), rna_annotation, fmt='%s')
mmwrite(path.join(rna_path, 'counts.mtx'), rna_counts, field = 'integer')

## Save multiome ATAC files

In [24]:
atac_counts = csr_matrix(adata_atac_mul.layers['counts'])
atac_features = adata_atac_mul.var_names
atac_cells = adata_atac_mul.obs_names
atac_annotation = np.array(adata_atac_mul.obs.cell_type)
atac_path = path.join(folder, 'multiome/ATAC')
np.savetxt(path.join(atac_path, 'cells.txt'), atac_cells, fmt='%s')
np.savetxt(path.join(atac_path, 'genes.txt'), atac_features, fmt='%s')
np.savetxt(path.join(atac_path, 'annotations.txt'), atac_annotation, fmt='%s')
mmwrite(path.join(atac_path, 'counts.mtx'), atac_counts, field = 'integer')

# Experimental design

In [47]:
seed = 10

## Subsample number of cells in RNA and ATAC

In [7]:
# paths
data_path = '/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC'
folders = ['RNA', 'ATAC']
ks = list(np.round(np.arange(0.2, 1., step=0.2), 1)) + [1,3,5,10,15]
for seed in range(20):
    random.seed(seed)
    np.random.seed(seed)
    for folder in folders:
        print('\n')
        for k in ks:
            print(seed, folder, k)
            cells = np.loadtxt(path.join(data_path, folder, 'cells.txt'), dtype=object, delimiter='SPACEISPARTOFTHENAME')
            cells_tmp = random.sample(list(cells), int(1000*k))
            savepath = path.join(data_path, folder, 'cells_%dk_seed%s.txt' % (k, seed) if k>=1 else 'cells_%.1fk_seed%s.txt' % (k, seed))
            print(savepath)
            print(len(cells), len(cells_tmp))
            np.savetxt(savepath, cells_tmp, fmt='%s')
            if folder == 'ATAC':
                savepath = path.join(data_path, 'ATAC.gene.activities', 'cells_%dk_seed%s.txt' % (k, seed) if k>=1 else 'cells_%.1fk_seed%s.txt' % (k, seed))
                np.savetxt(savepath, cells_tmp, fmt='%s')



0 RNA 0.2
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.2k_seed0.txt
26450 200
0 RNA 0.4
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.4k_seed0.txt
26450 400
0 RNA 0.6
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.6k_seed0.txt
26450 600
0 RNA 0.8
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.8k_seed0.txt
26450 800
0 RNA 1
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_1k_seed0.txt
26450 1000
0 RNA 3
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_3k_seed0.txt
26450 3000
0 RNA 5
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_5k_seed0.txt
26450 5000
0 RNA 10
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_10k_seed0.txt
26450 10000
0 RNA 15
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_15k_seed0.txt
26450 15000


0 ATAC 0.2
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark

/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_15k_seed4.txt
26450 15000


4 ATAC 0.2
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.2k_seed4.txt
22653 200
4 ATAC 0.4
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.4k_seed4.txt
22653 400
4 ATAC 0.6
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.6k_seed4.txt
22653 600
4 ATAC 0.8
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.8k_seed4.txt
22653 800
4 ATAC 1
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_1k_seed4.txt
22653 1000
4 ATAC 3
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_3k_seed4.txt
22653 3000
4 ATAC 5
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_5k_seed4.txt
22653 5000
4 ATAC 10
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_10k_seed4.txt
22653 10000
4 ATAC 15
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_bench

8 ATAC 5
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_5k_seed8.txt
22653 5000
8 ATAC 10
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_10k_seed8.txt
22653 10000
8 ATAC 15
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_15k_seed8.txt
22653 15000


9 RNA 0.2
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.2k_seed9.txt
26450 200
9 RNA 0.4
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.4k_seed9.txt
26450 400
9 RNA 0.6
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.6k_seed9.txt
26450 600
9 RNA 0.8
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.8k_seed9.txt
26450 800
9 RNA 1
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_1k_seed9.txt
26450 1000
9 RNA 3
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_3k_seed9.txt
26450 3000
9 RNA 5
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmar

/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_0.8k_seed13.txt
26450 800
13 RNA 1
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_1k_seed13.txt
26450 1000
13 RNA 3
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_3k_seed13.txt
26450 3000
13 RNA 5
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_5k_seed13.txt
26450 5000
13 RNA 10
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_10k_seed13.txt
26450 10000
13 RNA 15
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_15k_seed13.txt
26450 15000


13 ATAC 0.2
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.2k_seed13.txt
22653 200
13 ATAC 0.4
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.4k_seed13.txt
22653 400
13 ATAC 0.6
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.6k_seed13.txt
22653 600
13 ATAC 0.8
/gpfs/gibbs/pi/zhao/xs272/Multiomics/

/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA/cells_15k_seed17.txt
26450 15000


17 ATAC 0.2
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.2k_seed17.txt
22653 200
17 ATAC 0.4
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.4k_seed17.txt
22653 400
17 ATAC 0.6
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.6k_seed17.txt
22653 600
17 ATAC 0.8
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_0.8k_seed17.txt
22653 800
17 ATAC 1
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_1k_seed17.txt
22653 1000
17 ATAC 3
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_3k_seed17.txt
22653 3000
17 ATAC 5
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_5k_seed17.txt
22653 5000
17 ATAC 10
/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/ATAC/cells_10k_seed17.txt
22653 10000
17 ATAC 15
/gpfs/gibbs/pi/zhao/xs272/M

## Subsample number of multiome RNA and ATAC cells

In [10]:
# paths
data_path = '/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/multiome'
ks = list(np.round(np.arange(0.2, 1., step=0.2), 1)) + [1,3,5,10,15]
for seed in range(20):
    random.seed(seed)
    np.random.seed(seed)
    for k in ks:
        print(seed, k)
        cells = np.loadtxt(path.join(data_path, 'RNA', 'cells.txt'), dtype=object, delimiter='SPACEISPARTOFTHENAME')
        cells_tmp = random.sample(list(cells), int(1000*k))
        print(len(cells), len(cells_tmp))
        np.savetxt(path.join(data_path, 'RNA', 'cells_%dk_seed%s.txt' % (k, seed) if k>=1 else 'cells_%.1fk_seed%s.txt' % (k, seed)), cells_tmp, fmt='%s')
        np.savetxt(path.join(data_path, 'ATAC', 'cells_%dk_seed%s.txt' % (k, seed) if k>=1 else 'cells_%.1fk_seed%s.txt' % (k, seed)), cells_tmp, fmt='%s')


0 0.2
18467 200
0 0.4
18467 400
0 0.6
18467 600
0 0.8
18467 800
0 1
18467 1000
0 3
18467 3000
0 5
18467 5000
0 10
18467 10000
0 15
18467 15000
1 0.2
18467 200
1 0.4
18467 400
1 0.6
18467 600
1 0.8
18467 800
1 1
18467 1000
1 3
18467 3000
1 5
18467 5000
1 10
18467 10000
1 15
18467 15000
2 0.2
18467 200
2 0.4
18467 400
2 0.6
18467 600
2 0.8
18467 800
2 1
18467 1000
2 3
18467 3000
2 5
18467 5000
2 10
18467 10000
2 15
18467 15000
3 0.2
18467 200
3 0.4
18467 400
3 0.6
18467 600
3 0.8
18467 800
3 1
18467 1000
3 3
18467 3000
3 5
18467 5000
3 10
18467 10000
3 15
18467 15000
4 0.2
18467 200
4 0.4
18467 400
4 0.6
18467 600
4 0.8
18467 800
4 1
18467 1000
4 3
18467 3000
4 5
18467 5000
4 10
18467 10000
4 15
18467 15000
5 0.2
18467 200
5 0.4
18467 400
5 0.6
18467 600
5 0.8
18467 800
5 1
18467 1000
5 3
18467 3000
5 5
18467 5000
5 10
18467 10000
5 15
18467 15000
6 0.2
18467 200
6 0.4
18467 400
6 0.6
18467 600
6 0.8
18467 800
6 1
18467 1000
6 3
18467 3000
6 5
18467 5000
6 10
18467 10000
6 15
18467 15000

## Generate mislabelled RNA annotation txt files

In [13]:
data_path = '/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/RNA'
anno = np.loadtxt(path.join(data_path, 'annotations.txt'), dtype=object, delimiter='SPACEISPARTOFTHENAME')

In [14]:
dic = {}
for ct in set(anno):
    dic[ct] = np.sum(anno == ct)/len(anno)
prob = pd.DataFrame.from_dict(dic, orient='index')

In [15]:
def mislabel(anno, perc):
    idx = np.random.choice(len(anno), round(len(anno)*perc), replace=False)
    tmp = anno.copy()
    for i in idx:
        ct = anno[i]
        prob_ = prob[prob.index != ct]
        ct_ = prob_.index.tolist()
        prob_ = prob_.values.reshape(-1)
        tmp[i] = random.choices(ct_, prob_, k=1)[0]
    return tmp

In [ ]:
# # paths
# ks = [10,20,30,40,50,60,70,80,90,100]
# for seed in range(20):
#     random.seed(seed)
#     np.random.seed(seed)
#     for k in ks:
#         print(seed, k)
#         anno_tmp = mislabel(anno, k/100.)
#         print(np.sum(anno != anno_tmp) / len(anno))
#         np.savetxt(path.join(data_path, 'annotations_mis_percent_%d_seed%s.txt' % (k, seed)), anno_tmp, fmt='%s')

In [54]:
# paths
random.seed(10)
np.random.seed(10)
ks = [10,20,30,40,50,60,70,80,90,100]
for k in ks:
    print(k)
    anno_tmp = mislabel(anno, k/100.)
    print(np.sum(anno != anno_tmp) / len(anno))
    np.savetxt(path.join(data_path, 'annotations_mis_percent_%d.txt' % k), anno_tmp, fmt='%s')

1
0.009981096408317581
5
0.049981096408317584
10
0.1
20
0.2
30
0.3
40
0.4
50
0.5
60
0.6
70
0.7
80
0.8
90
0.9
100
1.0


## Generate cells.txt with partial cell tyes for RNA

In [31]:
data_path = '/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC'
cells = np.loadtxt(path.join(data_path, 'RNA', 'cells.txt'), dtype=object, delimiter='SPACEISPARTOFTHENAME')
anno = np.loadtxt(path.join(data_path, 'RNA', 'annotations.txt'), dtype=object, delimiter='SPACEISPARTOFTHENAME')

In [32]:
df = pd.DataFrame(anno, index=cells, columns=['anno'])

In [33]:
df.shape

(26450, 1)

In [34]:
df.anno.value_counts()

CD8+ T                 4267
CD14+ Mono             3404
NK                     3158
CD4+ T naive           2368
Naive CD20+ B          1965
CD4+ T activated       1685
Erythroblast           1237
Transitional B         1138
CD8+ T naive           1012
Lymph prog              682
CD16+ Mono              653
Proerythroblast         646
B1 B                    642
pDC                     632
ILC                     620
G/M prog                577
HSC                     542
cDC2                    384
Normoblast              375
MK/E prog               371
Plasma cell              51
ID2-hi myeloid prog      41
Name: anno, dtype: int64

In [35]:
tmp = df.anno.value_counts()
candidate_ct = list(tmp[(tmp < 1000) & (tmp > 200)].index)

In [36]:
candidate_ct

['Lymph prog',
 'CD16+ Mono',
 'Proerythroblast',
 'B1 B',
 'pDC',
 'ILC',
 'G/M prog',
 'HSC',
 'cDC2',
 'Normoblast',
 'MK/E prog']

In [37]:
# paths
ks = [2,4,6,8,10]
for seed in range(20):
    random.seed(seed)
    np.random.seed(seed)
    cells_10k = np.loadtxt(path.join(data_path, 'RNA', 'cells_10k_seed%s.txt' % seed), dtype=object, delimiter='SPACEISPARTOFTHENAME')
    for k in ks:
        print(seed, k)
        selected_ct = random.sample(candidate_ct, k)
        df_10k = df.loc[cells_10k]
        cells_tmp = list(df_10k[~df_10k.anno.isin(selected_ct)].index)
        savepath = path.join(data_path, 'RNA', 'cells_10k_%srm_seed%s.txt' % (k, seed))
        np.savetxt(savepath, cells_tmp, fmt='%s')

0 2
0 4
0 6
0 8
0 10
1 2
1 4
1 6
1 8
1 10
2 2
2 4
2 6
2 8
2 10
3 2
3 4
3 6
3 8
3 10
4 2
4 4
4 6
4 8
4 10
5 2
5 4
5 6
5 8
5 10
6 2
6 4
6 6
6 8
6 10
7 2
7 4
7 6
7 8
7 10
8 2
8 4
8 6
8 8
8 10
9 2
9 4
9 6
9 8
9 10
10 2
10 4
10 6
10 8
10 10
11 2
11 4
11 6
11 8
11 10
12 2
12 4
12 6
12 8
12 10
13 2
13 4
13 6
13 8
13 10
14 2
14 4
14 6
14 8
14 10
15 2
15 4
15 6
15 8
15 10
16 2
16 4
16 6
16 8
16 10
17 2
17 4
17 6
17 8
17 10
18 2
18 4
18 6
18 8
18 10
19 2
19 4
19 6
19 8
19 10


## Generate cells.txt with partial cell tyes for multiome

In [38]:
data_path = '/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/data/BMMC/multiome'
cells = np.loadtxt(path.join(data_path, 'RNA', 'cells.txt'), dtype=object, delimiter='SPACEISPARTOFTHENAME')
anno = np.loadtxt(path.join(data_path, 'RNA', 'annotations.txt'), dtype=object, delimiter='SPACEISPARTOFTHENAME')

In [39]:
df = pd.DataFrame(anno, index=cells, columns=['anno'])

In [40]:
df.shape

(18467, 1)

In [41]:
df.anno.value_counts()

CD14+ Mono             3243
NK                     1984
CD4+ T activated       1901
Erythroblast           1749
CD8+ T                 1653
Naive CD20+ B          1301
CD4+ T naive           1198
Transitional B          809
B1 B                    688
Normoblast              630
CD16+ Mono              526
Proerythroblast         472
Lymph prog              452
G/M prog                341
pDC                     338
MK/E prog               298
Plasma cell             286
HSC                     230
cDC2                    181
ILC                     120
ID2-hi myeloid prog      67
Name: anno, dtype: int64

In [42]:
tmp = df.anno.value_counts()
candidate_ct = list(tmp[(tmp < 800) & (tmp > 100)].index)

In [43]:
candidate_ct

['B1 B',
 'Normoblast',
 'CD16+ Mono',
 'Proerythroblast',
 'Lymph prog',
 'G/M prog',
 'pDC',
 'MK/E prog',
 'Plasma cell',
 'HSC',
 'cDC2',
 'ILC']

In [44]:
# paths
ks = [2,4,6,8,10]
for seed in range(20):
    random.seed(seed)
    np.random.seed(seed)
    cells_10k = np.loadtxt(path.join(data_path, 'RNA', 'cells_10k.txt'), dtype=object, delimiter='SPACEISPARTOFTHENAME')
    for k in ks:
        print(seed, k)
        selected_ct = random.sample(candidate_ct, k)
        df_10k = df.loc[cells_10k]
        cells_tmp = list(df_10k[~df_10k.anno.isin(selected_ct)].index)
        savepath = path.join(data_path, 'RNA', 'cells_10k_%srm_seed%s.txt' % (k, seed))
        np.savetxt(savepath, cells_tmp, fmt='%s')
        savepath = path.join(data_path, 'ATAC', 'cells_10k_%srm_seed%s.txt' % (k, seed))
        np.savetxt(savepath, cells_tmp, fmt='%s')

0 2
0 4
0 6
0 8
0 10
1 2
1 4
1 6
1 8
1 10
2 2
2 4
2 6
2 8
2 10
3 2
3 4
3 6
3 8
3 10
4 2
4 4
4 6
4 8
4 10
5 2
5 4
5 6
5 8
5 10
6 2
6 4
6 6
6 8
6 10
7 2
7 4
7 6
7 8
7 10
8 2
8 4
8 6
8 8
8 10
9 2
9 4
9 6
9 8
9 10
10 2
10 4
10 6
10 8
10 10
11 2
11 4
11 6
11 8
11 10
12 2
12 4
12 6
12 8
12 10
13 2
13 4
13 6
13 8
13 10
14 2
14 4
14 6
14 8
14 10
15 2
15 4
15 6
15 8
15 10
16 2
16 4
16 6
16 8
16 10
17 2
17 4
17 6
17 8
17 10
18 2
18 4
18 6
18 8
18 10
19 2
19 4
19 6
19 8
19 10


## Generate downsampled data matrix (switch kernel to R)

In [1]:
library(Matrix)
library(DropletUtils)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    

In [ ]:
data_path = '/gpfs/gibbs/pi/zhao/xs272/prep_pipeline/BMMC'

In [3]:
depth <- c(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9)
data_type <- c('RNA', 'ATAC', 'ATAC.gene.activities', 'multiome/RNA', 'multiome/ATAC')

In [2]:
for (d in data_type){
    count <- readMM(file.path(data_path, d, "counts.mtx"))
    for (p in depth){
        tmp <- DropletUtils::downsampleMatrix(t(count), prop=p)
        writeMM(t(tmp), file.path(data_path, d, paste0("counts_", p, ".mtx")))
    }
}

## Calculate unimodal ATAC gene activity (switch kernel to R)

In [1]:
setwd('/gpfs/gibbs/pi/zhao/xs272/prep_pipeline/')
library(Matrix)
library(readr)
source('/gpfs/gibbs/pi/zhao/xs272/data_preprocess/utils.r')
source('/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/scripts/utils.r')

Attaching SeuratObject

Attaching sp

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ensembldb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    

In [2]:
frag.path <- "/gpfs/gibbs/pi/zhao/yw599/Multiome/data/BMMC/post_competition/multiome"
frag.files <- c(
                s2d4="s2d4/atac_fragments.tsv.gz",
                s2d5="s2d5/atac_fragments.tsv.gz",
                s3d7="s3d7/atac_fragments.tsv.gz",
                s4d8="s4d8/atac_fragments.tsv.gz"
                )
barcode_suffix <- c(
                s2d4="s2d4",
                s2d5="6-s2d5",
                s3d7="12-s3d7",
                s4d8="14-s4d8")

In [3]:
# read previously processed data
single.atac.data <- load.data.folder('BMMC/ATAC/')
single.atac.count <- single.atac.data$count
single.atac.anno <- single.atac.data$annotations

In [4]:
# genome annotations
library(EnsDb.Hsapiens.v86) # hg38
library(Signac)
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotations) <- 'UCSC'

Warning message in .Seqinfo.mergexy(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .Seqinfo.mergexy(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .Seqinfo.mergexy(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .Seqinfo.mergexy(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .Seqinfo.mergexy(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .Seqinfo.mergexy(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .Seqinfo.mergexy(x, y):
"The 2 combined objec

In [5]:
batch <- sapply(colnames(single.atac.count), function(x) substr(x, (nchar(x) - 3), nchar(x)))

In [6]:
single.atac.ga.counts.list <- list()
for (tissue in names(frag.files)) {
    selected.id <- which(batch == tissue)
    selected.single.count <- single.atac.count[, selected.id]
    colnames(selected.single.count) <- gsub(barcode_suffix[tissue], '1', colnames(selected.single.count))
    selected.frag.path <- file.path(frag.path, frag.files[tissue])
    selected.ga <- convert.gene.activtity(selected.single.count, selected.frag.path, annotations, genome='hg38')
    colnames(selected.ga) <- gsub('1', barcode_suffix[tissue], colnames(selected.ga))
    single.atac.ga.counts.list[[tissue]] <- selected.ga
}

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from peaks to peaks_"
Extracting gene coordinates

Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from peaks to peaks_"
Extracting gene coordinates

Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from peaks to peaks_"
Extracting gene coordinates

Extracting reads overlapping genomic regions

Computing hash

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from peaks to peaks_"
Extracting gene coordinates

Extracting reads overlapping genomic regions



In [7]:
single.atac.ga.counts <- do.call(cbind, single.atac.ga.counts.list)

In [10]:
# Check cell barcodes
identical(colnames(single.atac.ga.counts), colnames(single.atac.count))

[1] TRUE

In [11]:
# Check features
print.names.match(rownames(single.atac.ga.counts), readLines('BMMC/RNA/genes.txt'))

[1] "Length1, Length2, Length intersection:"
[1] "19607 13431 11141"


In [13]:
# save data
write.counts.transpose(single.atac.ga.counts, 'BMMC/ATAC.gene.activities')
write(single.atac.anno, 'BMMC/ATAC.gene.activities/annotations.txt')

Warning message in dir.create(folder, recursive = TRUE):
"'BMMC/ATAC.gene.activities' already exists"
